In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Feb  8 15:45:44 2022

@author: ChewingGumKing_OJF
"""

#loads necessary libraries
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import os
from datetime import datetime
from datetime import date
import openpyxl
#import pyautogui
from selenium.webdriver.common.action_chains import ActionChains
#import shutil
#import glob
import os
import warnings
#import itertools
import json
import pandas as pd
import pyperclip as pc
from bs4 import BeautifulSoup as bs
import re
from random import randint
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
#import sys
warnings.filterwarnings("ignore")


In [3]:
driver=webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe')

In [6]:
url = "https://theilp.org.uk/events/"

In [7]:
driver.get(url)

In [367]:
soup = bs(driver.page_source,'lxml')

In [368]:
div = "ect-events-list-content" #class

In [369]:
section = soup.find('div',{'id':div})

In [370]:
each = section.find_all('div',{'class':'ect-list-post style-1 ect-simple-event future'})

In [355]:
theilp=[]
sidebar = "epta-sidebar-area" #div class
venue = "tribe-events-meta-group tribe-events-meta-group-venue" #div class
cost = "tribe-events-event-cost"#dd class
event_url ='tribe-events-event-url' #dd clas
email = "tribe-organizer-email" #dd class
for i in each:

    try:
        ven = i.find('div',{'class':'ect-list-venue default-venue'}).text.replace('\n\n',' ').replace('\n',' ').split('+')[0].strip()
    except:
        ven='Hosted Online'
        
    
    try:
        cost = i.find('div',{'class':'ect-rate-area'}).text.replace('\t','').replace('\n','')
    except:
        cost=''
    if cost=='':
        ticketlist=''
    elif cost=='Free':
        ty='Free'
        am=''
        cu=''
        ticketlist={'type': ty,
                     'price':am,
                     'currency': cu}
    elif '-' in cost:
        price=cost.split('-')[0][1:].replace(',','')
        ty='Paid'
        am=price
        cu=cost[0]
        ticketlist={'type': ty,
                     'price':am,
                     'currency': cu}
        
    else:
        ticketlist=''
    
    if ticketlist=='':
        ticket_list=ticketlist
    else:
        ticket_list=json.dumps(ticketlist)
    
    topic = i.h2.text.replace('\t','').replace('\n','')
    link = i.a['href']
#    data.append([topic,ven,cost,link])


    driver.get(link)
    time.sleep(10)
    soupn = bs(driver.page_source,'lxml')

    sidebar_ = soupn.find('div',{'class':sidebar})
    if str(sidebar_)!='None':
      #  print('aaa')
        try:
            address = sidebar_.find('div',{'class':"tribe-events-meta-group tribe-events-meta-group-venue"}).text.replace('Venue','').replace('\n\n\n','').replace('\n',' ').split('+')[0].strip()

        except:
            address = 'NA'
        try:
            web = sidebar_.find('dd',{'class':event_url}).text.strip()
            if len(url)<=5:
                web=link
        except:
            web = link
        try:
            email = sidebar_.find('dd',{'class':email}).text.strip()
            if len(email)<=5:
                email='NA'
        except:
            email = 'NA'
            
        #***********
        sidebar_ = soupn.find('div',{'class':'epta-sidebar-area'})
        sure=sidebar_.find('div',{'class':'tribe-events-meta-group tribe-events-meta-group-details'})
        title=soupn.find('div',{'class':'epta-title-date'}).h2.text.replace('\n','').replace('\t','')
        if 'Start' and 'End' in str(sure):
          #  print('schs')
            sak=sure.find_all('dd')[:2]
            siuu=[a.text.replace('\n','').replace('\t','').split('@') for a in sak]
            start,end=siuu
            yr=str(date.today().year)
            start=[[a.strip() for a in start][0]+' '+yr,[a.strip() for a in start][1].replace(' ','').replace('pm','PM').replace('am','AM')]
            end=[[a.strip() for a in end][0]+' '+yr,[a.strip() for a in end][1].replace(' ','').replace('pm','PM').replace('am','AM')]
            dat=[start[0],end[0]]
#            dat=[a.replace('st','').replace('nd','').replace('rd','').replace('th','').replace('Augu','August')]
            tim=[start[1],end[1],'']

            #siuu=[a.text.replace('\n','') for a in sak]
        elif 'Date' in str(sure):
           # print('uiuh')
            sak=sure.find_all('dd')[:2]
            siuu=[a.text.replace('\n','').replace('\t','').strip() for a in sak]
            yr=str(date.today().year)
            dat,tim=siuu
            if not re.search('\d{4}',dat):
                dat=[dat+' '+yr,dat+' '+yr]
#                dat=[a.replace('st','').replace('nd','').replace('rd','').replace('th','').replace('Augu','August')]
                tim=tim.replace(' ','').replace('pm','PM').replace('am','AM').split('-')+['']# for a in tim]
        else:
            title=topic
            dat=''
            tim=''
    else:
       # print('bbb')
        noside=soupn.find('div',{'class':'tribe-events-single tribe-blocks-editor'})
        title=noside.find('h1',{'class':'tribe-events-single-event-title'}).text
        timer=noside.find('h2',{'class':'tribe-events-schedule__datetime'}).get_text(separator='').replace('\t','').replace('\n','').replace('–','-').split('-')
        shi=[a.split('@') for a in timer]
        start,end=shi
        yr=str(date.today().year)
        start=[[a.strip() for a in start][0]+' '+yr,[a.strip() for a in start][1].replace(' ','').replace('pm','PM').replace('am','AM')]
        end=[[a.strip() for a in end][0]+' '+yr,[a.strip() for a in end][1].replace(' ','').replace('pm','PM').replace('am','AM')]
        if not re.search('\d{4}',start[0]) and re.search('\d{4}',start[1]):
            dat=[start[0],end[0]]
#            dat=[a.replace('st','').replace('nd','').replace('rd','').replace('th','').replace('Augu','August')]
            tim=[start[1],end[1],'']
        else:
            dat=''
            tim=''
        address=''
        web=link
        email=''
        
    #**************************
    if dat=='':
        start_date=end_date=''
    else:
        dat=[a.replace('st','').replace('nd','').replace('rd','').replace('th','').replace('Augu','August') for a in dat]
        spl_dt_obj = [datetime.strptime(v, '%d %B %Y') for v in dat]
        date_= [z.strftime('%Y-%m-%d') for z in spl_dt_obj]
        start_date=date_[0]
        end_date=date_[1]
        
    if tim=='':
        time_st=''
    else:
        time_st={'type': 'general',
                 'start_time':tim[0],
                 'end_time': tim[1],
                 'timezone': tim[2],
                 'days': 'all'}
    if email=='' or email=='NA':
        mail=' info@theilp.org.uk'
        mail_=json.dumps(mail)
    else:
        mail_=json.dumps(email)
    if time_st=='':
        time_=''
    else:
        time=json.dumps(time_st)
    event_web=web
    
     ###############COUNTYR,GPU#################
    def country(locale):
        try:
            google_url_for_location="https://www.google.com/search?q="+locale+"&oq="+locale+"&num=1"
            time.sleep(randint(0,3))
            driver.get(google_url_for_location)
            time.sleep(4)
            google_map_url=driver.find_element_by_id('lu_map').click()
            time.sleep(1)
            country=driver.find_element_by_class_name('x3AX1-LfntMc-header-title-VdSJob').text
            return(country)
        except Exception as e:
            print(e)
            return("")

    ####################################
    def get_google_map_url(location):
        try:
            google_url_for_location="https://www.google.com/search?q="+location+"&oq="+location+"&num=1"
            time.sleep(randint(0,3))
            driver.get(google_url_for_location)
            time.sleep(4)
            google_map_url=driver.find_element_by_id('lu_map').click()
            time.sleep(1)
            google_map_url=driver.current_url
            return(google_map_url)
        except Exception as e:
            print(e)
            return("")

    
    if 'Online' in ven:
        on_off='1'
        city=''
        country=''
        venue=''
        googlePlaceUrl=''
    else:
        on_off='0'
        city=ven
        venue=ven
        country=country(city)
        googlePlaceUrl=get_google_map_url(venue)
        
        
    org_pro=''
    org_name='Institution of Lighting Professionals'
    org_web='https://theilp.org.uk/'
    
    event_info=''    
    logo=''
    sponsors=''
    agendalist=''
    type_=''
    category=''
    Speakerlist=''
    
    theilp.append([link,title,start_date,end_date,time_,event_info,ticket_list,
                     org_pro,org_name,org_web,logo,sponsors,agendalist,
                     type_,category,city,country,venue,event_web,googlePlaceUrl,mail_,
                     Speakerlist,on_off])
print('Done')

aaa
uiuh
aaa
uiuh
bbb
aaa
uiuh
bbb
aaa
schs
aaa
schs
aaa
uiuh
aaa
schs
aaa
schs
aaa
schs
Done


In [ ]:
theilp_dict=theilp


theilp_df= pd.DataFrame(columns=['scrappedUrl','eventname','startdate','enddate','timing','eventinfo','ticketlist','orgProfile','orgName','orgWeb','logo','sponsor','agendalist','type','category','city','country','venue','event_website','googlePlaceUrl','ContactMail','Speakerlist','online_event'],data=theilp_dict)
theilp_df.to_csv("theilp.tsv", sep = '\t',index=False)


In [350]:
# start_date,end_date,time_st,address,web,email

('2022-04-25',
 '2022-04-25',
 {'type': 'general',
  'start_time': '1:00PM',
  'end_time': '3:00PM',
  'timezone': '',
  'days': 'all'},
 'NA',
 'https://www.eventbrite.co.uk/e/hi-lights-from-the-ilp-tickets-256708832027',
 'NA')

In [110]:
# driver.current_url

'https://theilp.org.uk/event/ilp-exterior-lighting-diploma-module-a-autumn-2022/'

In [293]:
# soupn = bs(driver.page_source,'lxml')
# noside=soupn.find('div',{'class':'tribe-events-single tribe-blocks-editor'})
# title=noside.find('h1',{'class':'tribe-events-single-event-title'}).text
# timer=noside.find('h2',{'class':'tribe-events-schedule__datetime'}).get_text(separator='').replace('\t','').replace('\n','').replace('–','-').split('-')
# shi=[a.split('@') for a in timer]
# start,end=shi
# yr=str(date.today().year)
# start=[[a.strip() for a in start][0]+' '+yr,[a.strip() for a in start][1].replace(' ','').replace('pm','PM').replace('am','AM')]
# end=[[a.strip() for a in end][0]+' '+yr,[a.strip() for a in end][1].replace(' ','').replace('pm','PM').replace('am','AM')]
# if not re.search('\d{4}',start[0]) and re.search('\d{4}',start[1]):
#     dat=[start[0],end[0]]
#     tim=[start[1],end[1],'']

AttributeError: 'NoneType' object has no attribute 'find'

In [144]:
# dat,tim

(['24th May 2022', '26th May 2022'], ['10:00AM', '5:00PM', ''])

In [315]:
# soupn = bs(driver.page_source,'lxml')
# sidebar_ = soupn.find('div',{'class':'epta-sidebar-area'})
# sure=sidebar_.find('div',{'class':'tribe-events-meta-group tribe-events-meta-group-details'})
# title=soupn.find('div',{'class':'epta-title-date'}).h2.text.replace('\n','').replace('\t','')
# if 'Start' and 'End' in str(sure):
#     sak=sure.find_all('dd')[:2]
#     siuu=[a.text.replace('\n','').replace('\t','').split('@') for a in sak]
#     start,end=siuu
#     yr=str(date.today().year)
#     start=[[a.strip() for a in start][0]+' '+yr,[a.strip() for a in start][1].replace(' ','').replace('pm','PM').replace('am','AM')]
#     end=[[a.strip() for a in end][0]+' '+yr,[a.strip() for a in end][1].replace(' ','').replace('pm','PM').replace('am','AM')]
#     dat=[start[0],end[0]]
#     tim=[start[1],end[1],'']
    
#     #siuu=[a.text.replace('\n','') for a in sak]
# elif 'Date' in str(sure):
#     sak=sure.find_all('dd')[:2]
#     siuu=[a.text.replace('\n','').replace('\t','').strip() for a in sak]
#     yr=str(date.today().year)
#     dat,tim=siuu
#     if not re.search('\d{4}',dat):
#         dat=[dat+' '+yr,dat+' '+yr]
#         tim=tim.replace(' ','').replace('pm','PM').replace('am','AM').split('-')+['']# for a in tim]
# else:
#     title=''
#     dat=''
#     tim=''